In [1]:
import ast

In [2]:
import ast
import keyword

class StringSearcher(ast.NodeVisitor):
    def __init__(self, target_string):
        self.target_string = target_string
        self.matches = []

    def generic_visit(self, node):
        # Check if the node has an attribute that could match the target string
        if isinstance(node, ast.Name) and node.id == self.target_string:
            self.matches.append(node)
        elif isinstance(node, ast.Attribute) and node.attr == self.target_string:
            self.matches.append(node)
        elif isinstance(node, ast.FunctionDef) and node.name == self.target_string:
            self.matches.append(node)
        elif isinstance(node, ast.ClassDef) and node.name == self.target_string:
            self.matches.append(node)
        # Add more conditions here if you want to check other node types
        
        super().generic_visit(node)

# Example usage
code = """
class MyClass:
    def __init__(self, value):
        self.value = value
    
    def my_method(self):
        return self.value

def my_function():
    pass

my_var = "Hello"
"""

# Parse the code into an AST
tree = ast.parse(code)

# Create the searcher and use it
searcher = StringSearcher("my_var")
searcher.visit(tree)

# Output the matches
for match in searcher.matches:
    print(f"Found a match: {type(match).__name__} at line {match.lineno}")


Found a match: Name at line 12


In [3]:
import ast

class CodeUsageFinder(ast.NodeVisitor):
    def __init__(self, name):
        self.name = name
        self.usages = []

    def visit_Name(self, node):
        if node.id == self.name:
            # When we find the name, we climb up the AST to find the statement that includes this name
            parent = node.parent
            while not isinstance(parent, ast.stmt):
                parent = parent.parent
            if parent not in self.usages:  # Avoid duplicates
                self.usages.append(parent)
        # Don't forget to traverse child nodes
        self.generic_visit(node)

    def visit(self, node):
        # Method override to add parent references dynamically
        for child in ast.iter_child_nodes(node):
            child.parent = node
        super().visit(node)

def ast_to_code(node):
    """Convert an AST node back to a string of code."""
    if isinstance(node, ast.Module):
        return "\n".join(ast_to_code(child) for child in node.body)
    elif isinstance(node, ast.FunctionDef):
        args = ", ".join(arg.arg for arg in node.args.args)
        return f"def {node.name}({args}):\n    " + "\n    ".join(ast_to_code(child) for child in node.body)
    elif isinstance(node, ast.Assign):
        targets = " = ".join(target.id for target in node.targets)
        return f"{targets} = {ast.unparse(node.value)}"
    elif isinstance(node, ast.Expr):
        return ast.unparse(node.value)
    elif isinstance(node, ast.Return):
        return f"return {ast.unparse(node.value)}"
    else:
        return ast.unparse(node)

code = """
def function(x, y):
    z = x + y
    return z

a = 5
b = function(a, 3)
"""

tree = ast.parse(code)
finder = CodeUsageFinder('a')
finder.visit(tree)

for usage in finder.usages:
    print(f"Code using 'a':\n{ast_to_code(usage)}\n")


Code using 'a':
a = 5

Code using 'a':
b = function(a, 3)

